# Task 1

In [4]:
from mysql.connector.pooling import MySQLConnectionPool
import mysql.connector as connector
from mysql.connector import Error
dbconfig = {
    "database":"litlle_lemon",
    "user" : "root",
    "password" : "server@1234"
}
p_name = 'pool_b'
p_size=2
try:
    pool = MySQLConnectionPool(
        pool_name = p_name,
        pool_size=p_size,
        host = 'localhost',
        **dbconfig
    )
    print("The connection pool is created with a name: ",p_name)
    print('The pool size is: ',p_size)
except Error as er:
    print("error: ", er)


The connection pool is created with a name:  pool_b
The pool size is:  2


# Task 2

In [14]:
guests = [
    {
        'Name': 'Guest 1',
        'Table Number': 8,
        'First Name': 'Anees',
        'Last Name': 'Java',
        'Booking Time': '18:00',
        'EmployeeID': 6
    },
    {
        'Name': 'Guest 2',
        'Table Number': 5,
        'First Name': 'Bald',
        'Last Name': 'Vin',
        'Booking Time': '19:00',
        'EmployeeID': 6
    },
    {
        'Name': 'Guest 3',
        'Table Number': 12,
        'First Name': 'Jay',
        'Last Name': 'Kon',
        'Booking Time': '19:30',
        'EmployeeID': 6
    }
]


In [16]:
insert_stmt = """
    INSERT INTO bookings (TableNo, GuestFirstName, GuestLastName, BookingSlot, EmployeeID) 
    VALUES (%s, %s, %s, %s, %s)
    """

In [15]:

for guest in guests:
    try:
        guest_connected = pool.get_connection()
        #print("[{}] is connected.\n".format(guest['Name']))
        if guest_connected.is_connected():
            cursor = guest_connected.cursor()
            values = (
                guest['Table Number'],
                guest['First Name'],
                guest['Last Name'],
                guest['Booking Time'],
                guest['EmployeeID']
            )
            cursor.execute(insert_stmt, values)
            guest_connected.commit()
            print("data added of {} ".format(guest['Name']))
    except:        
        connection=connector.connect(user="root",password="server@1234")
        pool.add_connection(cnx=connection)        
        user_connected = pool.get_connection()
        #print("[{}] is connected.\n".format(guest['Name']))
        if user_connected.is_connected():
            cursor = guest_connected.cursor()
            values = (
                guest['Table Number'],
                guest['First Name'],
                guest['Last Name'],
                guest['Booking Time'],
                guest['EmployeeID']
            )
            cursor.execute(insert_stmt, values)
            guest_connected.commit()
            print("data added of {} ".format(guest['Name']))
cursor.close()
guest_connected.close()
user_connected.close()

[Guest 1] is connected.

data added of Guest 1 
[Guest 2] is connected.

data added of Guest 2 
data added of Guest 3 


# Task 3

In [34]:
    try:
        connection = pool.get_connection()
        if connection.is_connected():
            cursor = connection.cursor()
            #task-3-part1
            cursor.execute("""
            select EmployeeID,Name from employees where Role = 'Manager';
            """)
            result = cursor.fetchall()
            for data in result:
                print("little lemon manger name is {} and ID is {}".format(data[1],data[0]))
            #task-3-part2    
            cursor.execute("""
            select Name,Role,Annual_Salary from employees where Annual_Salary =(select max(Annual_Salary) from employees);
            """)
            result2 =cursor.fetchall()
            for data in result2:
                print("\nThe {} having role of {} receives the highest salary of {}.".format(data[0],data[1],data[2]))
            #task-3-part3
            cursor.execute("""
            select count(BookingID) from bookings where BookingSlot between '18:00:00' and '20:00:00';
            """)
            result3 = cursor.fetchall()
            for data in result3:
                print("\nThere are {} guests booked between 18:00 and 20:00.".format(data[0]))
            #task-3-part4
            cursor.execute("""
            select b.BookingID as BookingID,concat(b.GuestFirstName," " ,b.GuestLastName) as FullName 
            from bookings b inner join employees e 
            on b.EmployeeID = e.EmployeeID 
            where e.Role ="Receptionist" order by 1;
            """)
            result4 = cursor.fetchall()
            print(cursor.column_names)
            for data in result4:
                print(data)
    except Error as er:
        print("error ", er)

cursor.close()
connection.close()

little lemon manger name is Mario Gollini and ID is 1

The Mario Gollini having role of Manager receives the highest salary of $70,000.

There are 7 guests booked between 18:00 and 20:00.
('BookingID', 'FullName')
(7, 'Anees Java')
(8, 'Bald Vin')
(9, 'Jay Kon')


# Task 4

In [1]:
BasicSalesReport_query = """
CREATE PROCEDURE BasicSalesReport()
BEGIN
select sum(BillAmount) as TotalSales, round(AVG(BillAmount),2) as AverageSale, 
min(BillAmount) as minbill, MAX(BillAmount) as maxbill
from orders;
END
"""

In [5]:
try:
    connection2 = pool.get_connection()
    if connection2.is_connected():
        cursor = connection2.cursor()
        cursor.execute(BasicSalesReport_query)
except Error as er:
    print("Error ",er)
    
cursor.close()
connection2.close()
    

In [14]:
try:
    connection2 = pool.get_connection()
    if connection2.is_connected():
        cursor = connection2.cursor()
        cursor.callproc('BasicSalesReport')
        result = next(cursor.stored_results())
        for colname in cursor.stored_results():
            colname = [col[0] for col in colname.description]
        result_ex = result.fetchall()
        for data in result_ex:
            print("Total Sales:{} \nAverage sale:{} \nMinimum Bill Paid:{} \nMinimum Bill Paid:{} ".
            format(data[0],data[1],data[2],data[3]))
except Error as er:
    print("Error: ",er)
    
cursor.close()
connection2.close()
    

Total Sales:243 
Average sale:48.60 
Minimum Bill Paid:37 
Minimum Bill Paid:86 


# Task 5

In [24]:
try:
    connection2 = pool.get_connection()
    if connection2.is_connected():
        cursor = connection2.cursor(buffered=True)
        cursor.execute("""
        select b.BookingSlot as BookingID,concat(b.GuestFirstName," " ,b.GuestLastName) as GuestName,
        e.Name as employename,e.Role as role
        from bookings b inner join employees e 
        on b.EmployeeID = e.EmployeeID 
        order by 1 asc limit 3;
        """)
        print(cursor.rowcount)
        result_ex = cursor.fetchall()
        for data in result_ex:
            print("{} \n{} \nAssigned to:{} [{}]\n ".
            format(data[0],data[1],data[2],data[3]))
except Error as er:
    print("Error: ",er)
    
cursor.close()
connection2.close()

3
15:00:00 
Vanessa McCarthy 
Assigned to:Giorgos Dioudis [Head Chef]
 
17:30:00 
Marcos Romero 
Assigned to:Fatma Kaya [Assistant Chef]
 
18:00:00 
Anees Java 
Assigned to:John Millar [Receptionist]
 


In [ ]:
con